# Guia do Projeto - Tech Challenge Fase 4
## Visão geral
Vamos construir um sistema que:
1. Lê um vídeo frame por frame (OpenCV)
2. Detecta rostos em cada frame (YOLOv11 - Ultralytics)
3. Analisa emoções de cada rosto (DeepFace)
4. Detecta atividades baseadas em pose (YOLOv11 Pose Estimation)
5. Gera um relatório com estatísticas (Markdown)

## Tecnologias Utilizadas
- **OpenCV**: Processamento de vídeo (leitura/gravação de frames)
- **YOLOv11 (Ultralytics)**: Detecção facial e pose estimation (múltiplas pessoas, oclusão, alta precisão)
- **DeepFace**: Análise de emoções
- **Python 3.13**: Linguagem principal


## Modelos de dados
Antes de processar, definimos quais dados vamos usar.

### Por que começar aqui?
Todo o código vai usar essas estruturas. Facilita entender o que cada função recebe e retorna.
O que precisamos representar?
1. BoundingBox: caixa que delimita um rosto (x, y, largura, altura)
2. FaceDetection: rosto detectado com sua posição e confiança
3. EmotionAnalysis: emoção detectada (feliz, triste, etc.)
4. FrameAnalysisResult: resultado da análise de um frame completo

In [7]:
from dataclasses import dataclass

@dataclass
class BoundingBox:
    x: int          # Coordenada X do canto superior esquerdo (pixels)
    y: int          # Coordenada Y do canto superior esquerdo (pixels)
    width: int      # Largura da caixa (pixels)
    height: int     # Altura da caixa (pixels)

# Exemplo de uso:
bbox = BoundingBox(x=100, y=50, width=200, height=300)
print(f"BoundingBox criada: {bbox}")
print(f"Posição: ({bbox.x}, {bbox.y}), Tamanho: {bbox.width}x{bbox.height}")

BoundingBox criada: BoundingBox(x=100, y=50, width=200, height=300)
Posição: (100, 50), Tamanho: 200x300


Explicação
- **x, y**: Posição do canto superior esquerdo da caixa (em pixels)
- **width, height**: Dimensões da caixa delimitadora (em pixels)
- **Uso**: Define onde um objeto (rosto, pessoa) está localizado na imagem

In [8]:
@dataclass
class FaceDetection:
    bounding_box: BoundingBox  # Onde está o rosto
    confidence: float          # Quanto confiamos (0.0 a 1.0)

bbox = BoundingBox(x=100, y=50, width=200, height=300)
face = FaceDetection(bounding_box=bbox, confidence=0.95)

print(f"Rosto detectado:")
print(f"  Posição: ({face.bounding_box.x}, {face.bounding_box.y})")
print(f"  Tamanho: {face.bounding_box.width}x{face.bounding_box.height}")
print(f"  Confiança: {face.confidence * 100:.1f}%")

Rosto detectado:
  Posição: (100, 50)
  Tamanho: 200x300
  Confiança: 95.0%


Explicação
- **bounding_box**: Usa o `BoundingBox` para definir onde o rosto está na imagem
- **confidence**: Nível de confiança da detecção (0.0 = sem confiança, 1.0 = total confiança)
- **Detecção**: Feita pelo **YOLOv11** (Ultralytics), que oferece:
  - Suporte para múltiplas pessoas simultâneas
  - Melhor precisão com oclusão parcial
  - Poses complexas
  - Compatibilidade com Python 3.13

In [9]:
@dataclass
class EmotionAnalysis:
    emotion: str      # Nome da emoção (ex: "happy", "sad", "neutral")
    confidence: float # Quanto confiamos nessa emoção (0.0 a 1.0)

emotion = EmotionAnalysis(emotion="happy", confidence=0.87)
print(f"Emoção detectada: {emotion.emotion}")
print(f"Confiança: {emotion.confidence * 100:.1f}%")

# Emoções possíveis:
emotions_list = [
    "happy", "sad", "angry", "surprise", 
    "neutral", "fear", "disgust"
]
print(f"\nEmoções suportadas: {', '.join(emotions_list)}")

Emoção detectada: happy
Confiança: 87.0%

Emoções suportadas: happy, sad, angry, surprise, neutral, fear, disgust


Explicação
- **emotion**: String com o nome da emoção detectada. Valores possíveis:
  - `"happy"` (feliz)
  - `"sad"` (triste)
  - `"angry"` (raiva)
  - `"surprise"` (surpresa)
  - `"neutral"` (neutro)
  - `"fear"` (medo)
  - `"disgust"` (nojo)
- **confidence**: Nível de confiança da detecção (0.0 a 1.0)
- **Análise**: Feita pelo **DeepFace**, que usa CNNs pré-treinadas para reconhecimento de emoções

In [10]:
@dataclass
class ActivityDetection:
    activity: str
    confidence: float
    

activity = ActivityDetection(activity="hands_up", confidence=0.92)
print(f"Atividade detectada: {activity.activity}")
print(f"Confiança: {activity.confidence * 100:.1f}%")

# Atividades comuns:
activities_list = [
    "hands_up",      # Mãos para cima
    "sitting",       # Sentado
    "standing",      # Em pé
    "walking",       # Caminhando
    "raising_hand",  # Levantando a mão
    "pointing"       # Apontando
]
print(f"\nAtividades suportadas: {', '.join(activities_list)}")

Atividade detectada: hands_up
Confiança: 92.0%

Atividades suportadas: hands_up, sitting, standing, walking, raising_hand, pointing


Explicação

Representa uma atividade detectada no vídeo baseada na **pose estimation** (análise de pontos corporais).

**Como funciona?**
1. O **YOLOv11** detecta pontos corporais (ombros, cotovelos, punhos, joelhos, etc.)
2. Analisa a posição relativa desses pontos
3. Infere ações baseadas em padrões (ex: se punhos estão acima dos ombros → "hands_up")

**Vantagens do YOLOv11 para atividades:**
- Detecta **múltiplas pessoas** simultaneamente
- Funciona mesmo com **oclusão parcial** (pessoa parcialmente escondida)
- Suporta **poses complexas**
- Melhor precisão que MediaPipe em cenários reais

**Atributos:**
- **activity**: Nome da atividade (ex: "hands_up", "sitting", "standing")
- **confidence**: Nível de confiança (0.0 a 1.0)



## Fase 1: VideoProcessor — leitura de vídeo
Conceitos fundamentais
#### 1. O que é um vídeo?
* Sequência de imagens (frames) exibidas rapidamente
* Exemplo: 30 FPS = 30 imagens por segundo
* Para processar, lemos frame por frame
#### 2. O que o VideoProcessor faz?
* Abre o arquivo de vídeo
* Lê cada frame individualmente
* Fornece informações (FPS, duração, resolução)
* Permite iterar pelos frames
#### 3. Como funciona com OpenCV?
* cv2.VideoCapture: abre o vídeo
* cap.read(): lê o próximo frame
* cap.get(): obtém propriedades (FPS, largura, altura, etc.)


In [11]:
import cv2
import os
from typing import Generator, Tuple
import logging

logger = logging.getLogger(__name__)

class VideoProcessor:
    def __init__(self, video_path: str):
        # Passo 1: Estrutura básica da classe
        # Validar se o arquivo existe
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"Video file not found: {video_path}")
        
        self.video_path = video_path
        # Abrir o vídeo
        self.cap = cv2.VideoCapture(video_path)
        
        # Verificar se abriu com sucesso
        if not self.cap.isOpened():
            raise ValueError(f"Could not open video: {video_path}")
        
        # Passo 2: Obter informações do vídeo
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        logger.info(
            f"Video loaded: {self.width}x{self.height} @ "
            f"{self.fps}fps, {self.frame_count} frames"
        )
    
    # Passo 3: Método para ler frames (generator)
    def get_frames(self) -> Generator[Tuple[int, float, cv2.Mat], None, None]:
        """
        Gera frames do vídeo um por um.
        
        Yields:
            Tuple[int, float, cv2.Mat]: (frame_number, timestamp, frame)
                - frame_number: número do frame (0, 1, 2, ...)
                - timestamp: tempo em segundos desde o início
                - frame: imagem do frame (numpy array)
        """
        frame_num = 0
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break  # Fim do vídeo
            
            timestamp = frame_num / self.fps
            yield frame_num, timestamp, frame
            frame_num += 1
            
    # Passo 4: Métodos auxiliares
    def release(self):
        """Libera os recursos do vídeo."""
        self.cap.release()
    
    def get_video_info(self) -> dict:
        """Retorna informações do vídeo."""
        return {
            "fps": self.fps,
            "frame_count": self.frame_count,
            "width": self.width,
            "height": self.height,
            "duration": self.frame_count / self.fps if self.fps > 0 else 0
        }

Explicação:
* ____init____: 
    * valida o arquivo e abre o vídeo
    * cv2.VideoCapture: objeto que representa o vídeo
    * isOpened(): verifica se o vídeo foi aberto corretamente
    * CAP_PROP_FPS: frames por segundo
    * CAP_PROP_FRAME_COUNT: total de frames
    * CAP_PROP_FRAME_WIDTH/HEIGHT: resolução

* get_frames: 
    * Generator (yield): retorna um frame por vez, sem carregar tudo na memória
    * cap.read(): retorna (ret, frame); ret indica sucesso
    * timestamp: tempo em segundos (frame_num / fps)

In [12]:
# Criar o processador
processor = VideoProcessor("meu_video.mp4")

# Ver informações
info = processor.get_video_info()
print(f"Vídeo: {info['width']}x{info['height']}")
print(f"FPS: {info['fps']}")
print(f"Duração: {info['duration']:.2f} segundos")

# Processar cada frame
for frame_num, timestamp, frame in processor.get_frames():
    print(f"Frame {frame_num} em {timestamp:.2f}s")
    # Aqui você processaria o frame (detecção, etc.)

# Liberar recursos
processor.release()

Vídeo: 1280x720
FPS: 30.0
Duração: 110.87 segundos


Frame 0 em 0.00s
Frame 1 em 0.03s
Frame 2 em 0.07s
Frame 3 em 0.10s
Frame 4 em 0.13s
Frame 5 em 0.17s
Frame 6 em 0.20s
Frame 7 em 0.23s
Frame 8 em 0.27s
Frame 9 em 0.30s
Frame 10 em 0.33s
Frame 11 em 0.37s
Frame 12 em 0.40s
Frame 13 em 0.43s
Frame 14 em 0.47s
Frame 15 em 0.50s
Frame 16 em 0.53s
Frame 17 em 0.57s
Frame 18 em 0.60s
Frame 19 em 0.63s
Frame 20 em 0.67s
Frame 21 em 0.70s
Frame 22 em 0.73s
Frame 23 em 0.77s
Frame 24 em 0.80s
Frame 25 em 0.83s
Frame 26 em 0.87s
Frame 27 em 0.90s
Frame 28 em 0.93s
Frame 29 em 0.97s
Frame 30 em 1.00s
Frame 31 em 1.03s
Frame 32 em 1.07s
Frame 33 em 1.10s
Frame 34 em 1.13s
Frame 35 em 1.17s
Frame 36 em 1.20s
Frame 37 em 1.23s
Frame 38 em 1.27s
Frame 39 em 1.30s
Frame 40 em 1.33s
Frame 41 em 1.37s
Frame 42 em 1.40s
Frame 43 em 1.43s
Frame 44 em 1.47s
Frame 45 em 1.50s
Frame 46 em 1.53s
Frame 47 em 1.57s
Frame 48 em 1.60s
Frame 49 em 1.63s
Frame 50 em 1.67s
Frame 51 em 1.70s
Frame 52 em 1.73s
Frame 53 em 1.77s
Frame 54 em 1.80s
Frame 55 em 1.83s
Fr